In [24]:
import os
from random import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import timedelta
import psycopg2
import tidalUtl.InterfaceSQL as isq
import tidalUtl.PrpUtl as prp

In [25]:
#config
DBNAME = "Splatoon"

In [15]:
mean_A4level = train['a4_level'].mean()
mean_B3level = train['b3_level'].mean()
mean_B4level = train['b4_level'].mean()
train = FillnaAndInsertIsnan(train, 
                             [('a1_rank','na'), ('a2_rank','na'), ('a3_rank','na'), ('a4_rank','na'), ('a4_weapon','na'), 
                              ('a4_level',mean_A4level), ('b1_rank','na'), ('b2_rank','na'), ('b3_rank','na'), ('b4_rank','na'),
                              ('b3_weapon','na'), ('b3_level',mean_B3level), ('b4_weapon','na'), ('b4_level',mean_B4level)])

In [16]:
mean_A4level = test['a4_level'].mean()
mean_B3level = test['b3_level'].mean()
mean_B4level = test['b4_level'].mean()
test = FillnaAndInsertIsnan(test, 
                             [('a1_rank','na'), ('a2_rank','na'), ('a3_rank','na'), ('a4_rank','na'), ('a4_weapon','na'), 
                              ('a4_level',mean_A4level), ('b1_rank','na'), ('b2_rank','na'), ('b3_rank','na'), ('b4_rank','na'),
                              ('b3_weapon','na'), ('b3_level',mean_B3level), ('b4_weapon','na'), ('b4_level',mean_B4level)])

In [17]:
train

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,a4_weapon_isnan,a4_level_isnan,b1_rank_isnan,b2_rank_isnan,b3_rank_isnan,b4_rank_isnan,b3_weapon_isnan,b3_level_isnan,b4_weapon_isnan,b4_level_isnan
0,1,regular,nawabari,sumeshi,sshooter_becchu,na,139,soytuber_custom,na,118,...,False,False,True,True,True,True,False,False,False,False
1,2,regular,nawabari,arowana,parashelter_sorella,na,198,jetsweeper,na,77,...,False,False,True,True,True,True,False,False,False,False
2,3,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,...,False,False,False,False,False,False,False,False,False,False
3,4,regular,nawabari,engawa,bamboo14mk1,na,336,splatroller_becchu,na,131,...,False,False,True,True,True,True,False,False,False,False
4,5,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,...,False,False,False,False,False,False,False,False,False,False
66121,66122,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,...,False,False,False,False,False,False,False,False,False,False
66122,66123,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,...,False,False,False,False,False,False,False,False,False,False
66123,66124,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,...,False,False,False,False,False,False,False,False,False,False


In [18]:
test

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,a4_weapon_isnan,a4_level_isnan,b1_rank_isnan,b2_rank_isnan,b3_rank_isnan,b4_rank_isnan,b3_weapon_isnan,b3_level_isnan,b4_weapon_isnan,b4_level_isnan
0,1,gachi,area,hakofugu,prime_collabo,x,174,herospinner_replica,x,130,...,False,False,False,False,False,False,False,False,False,False
1,2,gachi,asari,anchovy,prime_becchu,s+,363,nzap89,s+,59,...,False,False,False,False,False,False,False,False,False,False
2,3,gachi,area,mutsugoro,furo_deco,s,116,bold,s,125,...,False,False,False,False,False,False,False,False,False,False
3,4,gachi,asari,devon,prime_becchu,s+,192,splatspinner_collabo,s+,115,...,False,False,False,False,False,False,False,False,False,False
4,5,gachi,yagura,anchovy,l3reelgun_d,x,267,rapid_becchu,x,223,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,28336,gachi,yagura,battera,prime_becchu,x,67,splatroller,x,62,...,False,False,False,False,False,False,False,False,False,False
28336,28337,gachi,yagura,anchovy,longblaster_necro,c-,25,splatroller_collabo,c-,18,...,False,False,False,False,False,False,False,False,False,False
28337,28338,gachi,yagura,zatou,furo,x,386,kugelschreiber,x,136,...,False,False,False,False,False,False,False,False,False,False
28338,28339,gachi,asari,hokke,maneuver_becchu,s+,185,wakaba,s+,103,...,False,False,False,False,False,False,False,False,False,False


# Create ExecData

In [5]:
isq.createExecData(DBNAME)

DONE.


In [6]:
#すでに作られている場合はエラーが返る。
isq.createExecData(DBNAME)

DuplicateTable: リレーション"execdata_train"はすでに存在します


# Read ExecData

In [5]:
#全table
df = isq.selectExecData(DBNAME, "execdata_train", clmns = None)

In [6]:
df

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour,y
0,1,regular,nawabari,sumeshi,sshooter_becchu,None,139,soytuber_custom,None,118,...,None,68,sharp_neo,None,31,2019-10-15,20:00:00,78,20,1
1,2,regular,nawabari,arowana,parashelter_sorella,None,198,jetsweeper,None,77,...,None,168,sputtery_clear,None,151,2019-12-14,04:00:00,18,4,0
2,3,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,...,a-,160,dualsweeper_custom,a,126,2019-12-25,14:00:00,7,14,0
3,4,regular,nawabari,engawa,bamboo14mk1,None,336,splatroller_becchu,None,131,...,None,194,hotblaster_custom,None,391,2019-11-11,14:00:00,51,14,0
4,5,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,...,x,246,wakaba,x,160,2019-12-14,06:00:00,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,...,a,38,wakaba,a+,283,2019-10-11,10:00:00,82,10,1
66121,66122,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,...,s+,188,nzap89,s+,170,2019-12-04,00:00:00,28,0,1
66122,66123,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,...,a+,191,hissen_hue,a+,139,2019-10-22,00:00:00,71,0,1
66123,66124,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,...,s+,223,sharp_neo,s+,194,2019-12-11,00:00:00,21,0,0


In [7]:
#まず列をロード。
col = isq.readColumns(DBNAME, "execdata_train")
#col指定
col = col[col.cols != "id"].reset_index()
df = isq.selectExecData(DBNAME, "execdata_train", clmns = col)

In [8]:
df

,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,a3_weapon,...,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour,y
0,regular,nawabari,sumeshi,sshooter_becchu,None,139,soytuber_custom,None,118,pablo_hue,...,None,68,sharp_neo,None,31,2019-10-15,20:00:00,78,20,1
1,regular,nawabari,arowana,parashelter_sorella,None,198,jetsweeper,None,77,campingshelter_camo,...,None,168,sputtery_clear,None,151,2019-12-14,04:00:00,18,4,0
2,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,prime_becchu,...,a-,160,dualsweeper_custom,a,126,2019-12-25,14:00:00,7,14,0
3,regular,nawabari,engawa,bamboo14mk1,None,336,splatroller_becchu,None,131,dynamo_tesla,...,None,194,hotblaster_custom,None,391,2019-11-11,14:00:00,51,14,0
4,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,h3reelgun_d,...,x,246,wakaba,x,160,2019-12-14,06:00:00,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,wakaba,...,a,38,wakaba,a+,283,2019-10-11,10:00:00,82,10,1
66121,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,prime_becchu,...,s+,188,nzap89,s+,170,2019-12-04,00:00:00,28,0,1
66122,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,hissen,...,a+,191,hissen_hue,a+,139,2019-10-22,00:00:00,71,0,1
66123,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,dualsweeper_custom,...,s+,223,sharp_neo,s+,194,2019-12-11,00:00:00,21,0,0


In [9]:
#別例
col = isq.readColumns(DBNAME, "execdata_train")
#検索機能を使って当てはまるものを抽出。
col = col[(col["cols"].str.contains('a[0-9]_')) | (col["cols"].str.contains('b[0-9]_'))].reset_index()
df = isq.selectExecData(DBNAME, "execdata_train", clmns = col)

In [10]:
df

,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,a3_weapon,a3_rank,a3_level,a4_weapon,...,b1_level,b2_weapon,b2_rank,b2_level,b3_weapon,b3_rank,b3_level,b4_weapon,b4_rank,b4_level
0,sshooter_becchu,None,139,soytuber_custom,None,118,pablo_hue,None,13,hokusai,...,28,hokusai_becchu,None,26,herocharger_replica,None,68,sharp_neo,None,31
1,parashelter_sorella,None,198,jetsweeper,None,77,campingshelter_camo,None,198,nzap85,...,83,squiclean_b,None,118,campingshelter,None,168,sputtery_clear,None,151
2,nzap89,a-,114,quadhopper_black,a,68,prime_becchu,a,225,jetsweeper,...,50,nzap85,a+,163,prime_becchu,a-,160,dualsweeper_custom,a,126
3,bamboo14mk1,None,336,splatroller_becchu,None,131,dynamo_tesla,None,189,prime_becchu,...,273,liter4k,None,189,promodeler_mg,None,194,hotblaster_custom,None,391
4,bold_7,x,299,hissen_hue,x,97,h3reelgun_d,x,96,splatroller,...,101,sputtery_hue,x,45,bucketslosher_soda,x,246,wakaba,x,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,hokusai_becchu,a,66,soytuber,a,113,wakaba,a+,114,nzap83,...,509,nzap85,a+,272,splatcharger,a,38,wakaba,a+,283
66121,hokusai_becchu,s+,204,prime_becchu,s+,96,prime_becchu,s+,112,momiji,...,163,prime_becchu,s+,167,pablo,s+,188,nzap89,s+,170
66122,prime_collabo,a+,116,promodeler_rg,a,86,hissen,a-,19,sshooter_becchu,...,205,bold_neo,a+,70,nzap85,a+,191,hissen_hue,a+,139
66123,heroroller_replica,s+,210,sharp_neo,s+,96,dualsweeper_custom,s+,126,prime_becchu,...,91,heroroller_replica,s+,395,rapid_elite_deco,s+,223,sharp_neo,s+,194


# Preprocessing: Update, Insert

In [6]:
#欠損値補完
#train
mean_A4level = train['a4_level'].mean()
mean_B3level = train['b3_level'].mean()
mean_B4level = train['b4_level'].mean()
train = prp.FillnaAndInsertIsnan(train, 
                             [('a1_rank','na'), ('a2_rank','na'), ('a3_rank','na'), ('a4_rank','na'), ('a4_weapon','na'), 
                              ('a4_level',mean_A4level), ('b1_rank','na'), ('b2_rank','na'), ('b3_rank','na'), ('b4_rank','na'),
                              ('b3_weapon','na'), ('b3_level',mean_B3level), ('b4_weapon','na'), ('b4_level',mean_B4level)])
#test
mean_A4level = test['a4_level'].mean()
mean_B3level = test['b3_level'].mean()
mean_B4level = test['b4_level'].mean()
test = prp.FillnaAndInsertIsnan(test, 
                             [('a1_rank','na'), ('a2_rank','na'), ('a3_rank','na'), ('a4_rank','na'), ('a4_weapon','na'), 
                              ('a4_level',mean_A4level), ('b1_rank','na'), ('b2_rank','na'), ('b3_rank','na'), ('b4_rank','na'),
                              ('b3_weapon','na'), ('b3_level',mean_B3level), ('b4_weapon','na'), ('b4_level',mean_B4level)])

NameError: name 'train' is not defined

In [26]:
#欠損値補完
#train
#まず列名をロード。
col = isq.readColumns(DBNAME, "execdata_train")
#欠損値のある列をロード
col = col[(col.cols == "a1_rank")   | (col.cols == "a2_rank") 
        | (col.cols == "a3_rank")   | (col.cols == "a4_rank") 
        | (col.cols == "a4_weapon") | (col.cols == "a4_level") 
        | (col.cols == "b1_rank")   | (col.cols == "b2_rank") 
        | (col.cols == "b3_rank")   | (col.cols == "b4_rank") 
        | (col.cols == "b3_weapon") | (col.cols == "b3_level") 
        | (col.cols == "b4_weapon") | (col.cols == "b4_level")].reset_index()
train = isq.selectExecData(DBNAME, "execdata_train", clmns = col)

#int値の欠損値：平均値
mean_A4level = int(train['a4_level'].mean())
mean_B3level = int(train['b3_level'].mean())
mean_B4level = int(train['b4_level'].mean())
#欠損値を補完
train, dfIsNan = prp.FillnaAndInsertIsnan(train, 
                             [('a1_rank','na'), ('a2_rank','na'), ('a3_rank','na'), ('a4_rank','na'), ('a4_weapon','na'), 
                              ('a4_level',mean_A4level), ('b1_rank','na'), ('b2_rank','na'), ('b3_rank','na'), ('b4_rank','na'),
                              ('b3_weapon','na'), ('b3_level',mean_B3level), ('b4_weapon','na'), ('b4_level',mean_B4level)])

#DBへUpdate & Insert

In [27]:
train

,a1_rank,a2_rank,a3_rank,a4_weapon,a4_rank,a4_level,b1_rank,b2_rank,b3_weapon,b3_rank,b3_level,b4_weapon,b4_rank,b4_level
0,na,na,na,hokusai,na,10,na,na,herocharger_replica,na,68,sharp_neo,na,31
1,na,na,na,nzap85,na,123,na,na,campingshelter,na,168,sputtery_clear,na,151
2,a-,a,a,jetsweeper,a,107,a-,a+,prime_becchu,a-,160,dualsweeper_custom,a,126
3,na,na,na,prime_becchu,na,41,na,na,promodeler_mg,na,194,hotblaster_custom,na,391
4,x,x,x,splatroller,x,136,x,x,bucketslosher_soda,x,246,wakaba,x,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,a,a,a+,nzap83,a+,123,a+,a+,splatcharger,a,38,wakaba,a+,283
66121,s+,s+,s+,momiji,s+,44,s+,s+,pablo,s+,188,nzap89,s+,170
66122,a+,a,a-,sshooter_becchu,a+,69,a+,a+,nzap85,a+,191,hissen_hue,a+,139
66123,s+,s+,s+,prime_becchu,s+,453,s+,s+,rapid_elite_deco,s+,223,sharp_neo,s+,194


In [28]:
dfIsNan

,a1_rank,a2_rank_isnan,a3_rank_isnan,a4_rank_isnan,a4_weapon_isnan,a4_level_isnan,b1_rank_isnan,b2_rank_isnan,b3_rank_isnan,b4_rank_isnan,b3_weapon_isnan,b3_level_isnan,b4_weapon_isnan,b4_level_isnan
0,True,True,True,True,False,False,True,True,True,True,False,False,False,False
1,True,True,True,True,False,False,True,True,True,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,True,True,True,False,False,True,True,True,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66121,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66122,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66123,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [29]:
train.isnull().sum()

a1_rank      0
a2_rank      0
a3_rank      0
a4_weapon    0
a4_rank      0
a4_level     0
b1_rank      0
b2_rank      0
b3_weapon    0
b3_rank      0
b3_level     0
b4_weapon    0
b4_rank      0
b4_level     0
dtype: int64